# About this notebook

- PyTorch NN starter code
- MultilabelStratifiedKFold 5 folds 

If this notebook is helpful, feel free to upvote :)

# Library

In [1]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import os
import gc
import random
import math
import time
from collections import defaultdict

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import log_loss

import category_encoders as ce

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import common as com
from MLP_Dataset import TrainDataset, TestDataset
from MLP_Model import FCBlock, TabularNN
from trainer import train_fn, validate_fn, inference_fn, AverageMeter
from config import CFG

# Utils

In [3]:
out_base_path = "/media/hiroki/working/kaggle/Mechanisms-of-Action-Prediction/result/MLP/max_out"
model_path = out_base_path + '/model'
os.makedirs(out_base_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

In [4]:
logger = com.get_logger(out_base_path+'/max_out')
com.seed_everything(seed=42)

# Data Loading

In [5]:
base_dir = '/media/hiroki/working/kaggle/Mechanisms-of-Action-Prediction/datasets'
#os.listdir('../input/lish-moa')
os.listdir(base_dir)

['sample_submission.csv',
 'test_features.csv',
 'train_features.csv',
 'train_targets_nonscored.csv',
 'train_targets_scored.csv']

In [6]:
train_features = pd.read_csv(base_dir + '/train_features.csv')
train_targets_scored = pd.read_csv(base_dir + '/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(base_dir + '/train_targets_nonscored.csv')
test_features = pd.read_csv(base_dir + '/test_features.csv')
submission = pd.read_csv(base_dir + '/sample_submission.csv')

#train_features = pd.read_csv('../input/lish-moa/train_features.csv')
#train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
#train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
#test_features = pd.read_csv('../input/lish-moa/test_features.csv')
#submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [7]:
# ref: https://www.kaggle.com/c/lish-moa/discussion/180165
# check if labels for 'ctl_vehicle' are all 0.
train = train_features.merge(train_targets_scored, on='sig_id')
target_cols = [c for c in train_targets_scored.columns if c not in ['sig_id']]
cols = target_cols + ['cp_type']
train[cols].groupby('cp_type').sum().sum(1)

cp_type
ctl_vehicle        0
trt_cp         16844
dtype: int64

- labels for 'ctl_vehicle' are all 0.

In [8]:
# constrcut train&test except 'cp_type'=='ctl_vehicle' data
print(train_features.shape, test_features.shape)
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
print(train.shape, test.shape)

(23814, 876) (3982, 876)
(21948, 1082) (3624, 876)


# CV split

In [9]:
folds = train.copy()
Fold = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[target_cols])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.shape)

(21948, 1083)


# Dataset

In [10]:
cat_features = ['cp_time', 'cp_dose']
num_features = [c for c in train.columns if train.dtypes[c] != 'object']
num_features = [c for c in num_features if c not in cat_features]
num_features = [c for c in num_features if c not in target_cols]
target = train[target_cols].values

def cate2num(df):
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72: 2})
    df['cp_dose'] = df['cp_dose'].map({'D1': 3, 'D2': 4})
    return df

train = cate2num(train)
test = cate2num(test)

# MODEL

In [11]:
def run_single_nn(cfg, train, test, folds, num_features, cat_features, target, target_cols, device, model_out_path, fold_num=0, seed=42):
    
    # Set seed
    logger.info(f'Set seed {seed}')
    com.seed_everything(seed=seed)

    # loader
    trn_idx = folds[folds['fold'] != fold_num].index
    val_idx = folds[folds['fold'] == fold_num].index
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    train_target = target[trn_idx]
    valid_target = target[val_idx]
    train_dataset = TrainDataset(train_folds, num_features, cat_features, train_target)
    valid_dataset = TrainDataset(valid_folds, num_features, cat_features, valid_target)
    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, 
                              num_workers=4, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.batch_size, shuffle=False, 
                              num_workers=4, pin_memory=True, drop_last=False)

    # model
    model = TabularNN(cfg, num_features, target_cols)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=cfg.epochs, steps_per_epoch=len(train_loader))

    # log
    log_df = pd.DataFrame(columns=(['EPOCH']+['TRAIN_LOSS']+['VALID_LOSS']) )

    # train & validate
    best_loss = np.inf
    early_stopping_cnt = 0
    for epoch in range(cfg.epochs):
        train_loss = train_fn(train_loader, model, optimizer, epoch, scheduler, device)
        valid_loss, val_preds = validate_fn(valid_loader, model, device)
        log_row = {'EPOCH': epoch, 
                   'TRAIN_LOSS': train_loss,
                   'VALID_LOSS': valid_loss,
                  }
        log_df = log_df.append(pd.DataFrame(log_row, index=[0]), sort=False)
        #logger.info(log_df.tail(1))
        if valid_loss < best_loss:
            logger.info(f'epoch{epoch} save best model... tr_loss:{train_loss}, val_loss{valid_loss}')
            best_loss = valid_loss
            oof = np.zeros((len(train), len(target_cols)))
            oof[val_idx] = val_preds
            torch.save(model.state_dict(), z + f"/fold{fold_num}_seed{seed}.pth")
            best_epoch = epoch
            early_stopping_cnt = 0
        else:
            early_stopping_cnt += 1
            if early_stopping_cnt == cfg.early_stopping_rounds:
                logger.info(f'best epoch: epoch{best_epoch}')
                break

    # predictions
    test_dataset = TestDataset(test, num_features, cat_features)
    test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, 
                             num_workers=4, pin_memory=True)
    model = TabularNN(cfg, num_features, target_cols)
    model.load_state_dict(torch.load(model_out_path + f"/fold{fold_num}_seed{seed}.pth"))
    model.to(device)
    predictions = inference_fn(test_loader, model, device)
    
    # del
    torch.cuda.empty_cache()

    return oof, predictions, log_df


def run_kfold_nn(cfg, train, test, folds, num_features, cat_features, target, target_cols, device, model_out_path, n_fold=5, seed=42):

    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    log_dfs = []
    for _fold in range(n_fold):
        logger.info("Fold {}".format(_fold))
        _oof, _predictions, log_df = run_single_nn(cfg,
                                                   train,
                                                   test,
                                                   folds,
                                                   num_features, 
                                                   cat_features,
                                                   target,
                                                   target_cols,
                                                   device,
                                                   model_out_path,
                                                   fold_num=_fold,
                                                   seed=seed)
        oof += _oof
        predictions += _predictions / n_fold
        log_dfs.append(log_df)

    score = 0
    for i in range(target.shape[1]):
        _score = log_loss(target[:,i], oof[:,i])
        score += _score / target.shape[1]
    logger.info(f"CV score: {score}")
    
    return oof, predictions, log_dfs

In [12]:
# Seed Averaging for solid result
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

#SEED = [0, 1, 2]
#for seed in SEED:
oof, predictions, log_dfs = run_kfold_nn(CFG,
                                         train, test, folds, 
                                         num_features, cat_features, target, target_cols,
                                         device,
                                         model_path=model_path,
                                         n_fold=5, seed=42)
score = 0
for i in range(target.shape[1]):
    _score = log_loss(target[:,i], oof[:,i])
    score += _score / target.shape[1]
#logger.info(f"Seed Averaged CV score: {score}")

Fold 0
Set seed 42
epoch0 save best model... tr_loss:0.9240619543477566, val_loss0.6732538875794899
epoch1 save best model... tr_loss:0.6889511928723676, val_loss0.550060255462325
epoch2 save best model... tr_loss:0.5270443650057716, val_loss0.4268673354781027
epoch3 save best model... tr_loss:0.38665683556647196, val_loss0.3298621796665539
epoch4 save best model... tr_loss:0.2655657759602488, val_loss0.25701901533880644
epoch5 save best model... tr_loss:0.18476831189689846, val_loss0.20601311005601036
epoch6 save best model... tr_loss:0.13241072749569469, val_loss0.16203171984888698
epoch7 save best model... tr_loss:0.0972167680271133, val_loss0.1304065280324506
epoch8 save best model... tr_loss:0.07369995562454862, val_loss0.10674398455234214
epoch9 save best model... tr_loss:0.05760134778998412, val_loss0.08790780342860754
epoch10 save best model... tr_loss:0.04622802676728172, val_loss0.07207451391803919
epoch11 save best model... tr_loss:0.03843873908512131, val_loss0.062098477482

epoch37 save best model... tr_loss:0.01720890138862505, val_loss0.01940485894221787
epoch38 save best model... tr_loss:0.017056234619396662, val_loss0.019378004285903618
epoch39 save best model... tr_loss:0.016914812470684305, val_loss0.019303466052088488
epoch40 save best model... tr_loss:0.016754702846757577, val_loss0.019253449514508247
epoch41 save best model... tr_loss:0.016583311927239715, val_loss0.018805486417814647
epoch42 save best model... tr_loss:0.016430884970175308, val_loss0.018747710720010118
epoch43 save best model... tr_loss:0.01628223289803595, val_loss0.01870010435700009
epoch44 save best model... tr_loss:0.016164313809755835, val_loss0.018637379095942666
epoch45 save best model... tr_loss:0.015989141322903497, val_loss0.018528664095119354
epoch46 save best model... tr_loss:0.015845551639863283, val_loss0.018340601508159842
epoch48 save best model... tr_loss:0.015555702883199565, val_loss0.01828987473167971
epoch49 save best model... tr_loss:0.015400894171469948, va

epoch12 save best model... tr_loss:0.032993568933439736, val_loss0.05249629310228287
epoch13 save best model... tr_loss:0.02927556314104556, val_loss0.0470425704124984
epoch14 save best model... tr_loss:0.02664074614766414, val_loss0.03984545701287331
epoch15 save best model... tr_loss:0.02475022425004927, val_loss0.03594547318366774
epoch16 save best model... tr_loss:0.02342257162311325, val_loss0.03267226052515056
epoch17 save best model... tr_loss:0.022451545706657815, val_loss0.029924871237918836
epoch18 save best model... tr_loss:0.021779324235666517, val_loss0.02795870695984445
epoch19 save best model... tr_loss:0.021200164994496826, val_loss0.02663370419997968
epoch20 save best model... tr_loss:0.02072431195024265, val_loss0.025594968346195776
epoch21 save best model... tr_loss:0.020392942587321584, val_loss0.02425177165377113
epoch22 save best model... tr_loss:0.020085550552356418, val_loss0.023623351483756425
epoch23 save best model... tr_loss:0.019789210486694846, val_loss0.0

epoch64 save best model... tr_loss:0.0129826798173429, val_loss0.017163798183722195
epoch65 save best model... tr_loss:0.012869385971830492, val_loss0.01715147687008914
epoch66 save best model... tr_loss:0.012675781627219633, val_loss0.017054034329952505
epoch73 save best model... tr_loss:0.011563443731734135, val_loss0.016999378831160383
best epoch: epoch73
CV score: 0.016848603170290157


In [18]:
train[target_cols] = oof
train[['sig_id']+target_cols].to_csv(out_base_path + '/oof.csv', index=False)

test[target_cols] = predictions
test[['sig_id']+target_cols].to_csv(out_base_path + '/pred.csv', index=False)

In [19]:
# Final result with 'cp_type'=='ctl_vehicle' data
result = train_targets_scored.drop(columns=target_cols)\
            .merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
y_true = train_targets_scored[target_cols].values
y_pred = result[target_cols].values
score = 0
for i in range(y_true.shape[1]):
    _score = log_loss(y_true[:,i], y_pred[:,i])
    score += _score / y_true.shape[1]
logger.info(f"Final result: {score}")

Final result: 0.015528392642207555


# eval

In [15]:
#model_path='/media/hiroki/working/kaggle/Mechanisms-of-Action-Prediction/working/experiment/MLP/baseline2/fold0_seed0.pth'

In [20]:
def run_single_nn_evaluation(cfg, model_path, test, num_features, cat_features, target, device):
    # predictions
    test_dataset = TestDataset(test, num_features, cat_features)
    test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, 
                             num_workers=4, pin_memory=True)
    model = TabularNN(cfg)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    predictions = inference_fn(test_loader, model, device)
    
    # del
    torch.cuda.empty_cache()
    return predictions

In [21]:
# Seed Averaging for solid result
#oof = np.zeros((len(train), len(CFG.target_cols)))

prediction = run_single_nn_evaluation(cfg=CFG,
                                      model_path=model_path,
                                      test=test,
                                      num_features=num_features,
                                      cat_features=cat_features,
                                      target=target,
                                      device='cpu')

NameError: name 'model_path' is not defined

In [ ]:
test[target_cols] = predictions
test[['sig_id']+target_cols].to_csv('pred.csv', index=False)

# Submit

In [ ]:
sub = submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)
sub.head()